In [74]:
import arcpy
import pandas as pd
import numpy as np
from arcpy.sa import *

# 空间分配因子计算--道路分配

## 1. 判断每个网格都属于哪个行政区。

此步骤通过相交分析来实现。

输入CMAQ网格（small_grid）和行政区网格(big_grid)来进行相交。

**small_grid: CMAQ网格路径，此网格建议通过ISAT生成，此变量为全局变量。**

**big_grid: 行政区矢量文件路径，分配的单元大小以此矢量要素为准，此变量为全局变量。**

**out_path: 所有中间文件和最终文件都将在此目录下输出，此变量为全局变量。**

In [75]:
small_grid = 'E:\domain\Chengdu-3km\Domain_CMAQ\domain03_cmaq.shp'
big_grid = 'E:\全国矢量地图\国家基础地理信息系统\中国地州界.shp'
out_path = f'F:\github\Haofan_Emission_Tools\output_directory'

out_feature_class = f'{out_path}\cmaq_intersect.shp'
in_features = [small_grid, big_grid]
arcpy.analysis.Intersect(in_features, out_feature_class, "ALL")

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\cmaq_intersect.shp'>

**在相交的过程中可能会存在行政区边界将规则的CMAQ网格分割成多个小碎块的情况，此时就会出现多个重复ID，无法判断此ID所在的网格是属于哪个行政区。
因此下面的代码则是为了解决这种情况：如果该网格在A行政区中所占面积最大，那么则认为该网格属于A行政区。**

In [77]:
arcpy.CalculateGeometryAttributes_management(out_feature_class, [['square', 'AREA']], area_unit='SQUARE_KILOMETERS')
out_name = 'cmaq_intersect.csv'
arcpy.conversion.TableToTable(out_feature_class, out_path, out_name)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\cmaq_intersect.shp'>

In [79]:
# 此代码块则为删除面积占比较小的属性条
df = pd.read_csv(f'{out_path}/{out_name}')
df_ids = df['ID'].values
df_area = df['square'].values
del_list = []
for temp_id in range(1, df_ids.max()+1):
    pos_in_df = np.where(df_ids == temp_id)[0]
    temp_area = df_area[pos_in_df]
    if len(temp_area) > 1.0:
        pos_in_temp = np.where(temp_area != temp_area.max())
        pos = pos_in_df[pos_in_temp]
        for temp_pos in pos:
            del_list.append(temp_pos)
df = df.drop(del_list)
df

,OID_,ID,LON,LAT,NAME,square
0,0,1,101.576029,28.399021,康定县,9.000000
1,1,2,101.606922,28.399710,康定县,9.000000
2,2,3,101.637815,28.400390,康定县,9.000000
3,3,4,101.668708,28.401061,康定县,9.000000
4,4,5,101.699602,28.401724,康定县,9.000000
...,...,...,...,...,...,...
26776,26776,24645,106.391070,32.680985,汉中市,9.000000
26777,26777,24646,106.423582,32.680311,汉中市,9.000000
26778,26778,24647,106.456093,32.679627,汉中市,9.000000
26779,26779,24648,106.488604,32.678935,汉中市,7.737879


## 进行道路分配

### 计算每个行政区中的道路总长

In [80]:
# Clip
clipped_big_grid = f'{out_path}\Big_grid_cliped.shp'
arcpy.Clip_analysis(big_grid, small_grid, clipped_big_grid)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Big_grid_cliped.shp'>

In [89]:
# 从此处开始修改道路文件 可以进行多次分配 不用再运行以上的过程
road_file = r'E:\Model\build_emission\建立全国排放因子数据库\全国路网数据\2020年我国城市道路数据集\城市一级道路.shp'

out_feature_class = f'{out_path}\Big_grid_road.shp'
in_features = [road_file, clipped_big_grid]
arcpy.analysis.Intersect(in_features, out_feature_class, "ALL")

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Big_grid_road.shp'>

仅保留行政区名称字段（保证该字段名称为“NAME”）和必要字段。

In [90]:
arcpy.CalculateGeometryAttributes_management(out_feature_class, [["Length_m", "LENGTH_GEODESIC"],], "METERS")

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Big_grid_road.shp'>

In [91]:
out_statistic_table = f'{out_feature_class[0:-4]}.dbf'
out_path = f'F:\github\Haofan_Emission_Tools\output_directory'
out_name = 'Big_grid_road.csv'
arcpy.conversion.TableToTable(out_statistic_table, out_path, out_name)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Big_grid_road.csv'>

In [98]:
temp_big_grid_sst = pd.read_csv(f'{out_path}/{out_name}')
name_list = np.unique(temp_big_grid_sst['NAME'])
length_list = []
for temp_area in np.unique(temp_big_grid_sst['NAME']):
        temp_length = temp_big_grid_sst[temp_big_grid_sst['NAME'].isin([temp_area])]['Length_m'].values.sum()
        length_list.append(temp_length)
big_grid_sst = pd.DataFrame(columns=['NAME', 'LENGTH'])
big_grid_sst['NAME'] = name_list
big_grid_sst['LENGTH'] = length_list
big_grid_sst

,NAME,LENGTH
0,乐山市,1.927561e+05
1,仁寿县,1.703509e+05
2,内江市,1.439561e+05
3,华蓥市,2.229052e+04
4,南充县,3.387715e+05
5,宜宾市,2.377597e+05
6,广元市,5.299714e+04
7,德阳市,6.683858e+05
8,成都市,4.928331e+06
9,泸州市,1.532951e+05


### 计算每个小网格中的道路总长

In [99]:
in_features = [road_file, small_grid]
out_feature_class = f'F:\github\Haofan_Emission_Tools\output_directory\Small_grid_road.shp'

arcpy.analysis.Intersect(in_features, out_feature_class, "ALL")

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Small_grid_road.shp'>

仅保留ID字段（保证该字段名称为“ID”）和必要字段。

In [100]:
arcpy.CalculateGeometryAttributes_management(out_feature_class, [["Length_m", "LENGTH_GEODESIC"],], "METERS")

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Small_grid_road.shp'>

In [101]:
out_statistic_table = f'{out_feature_class[0:-4]}.dbf'
out_path = f'F:\github\Haofan_Emission_Tools\output_directory'
out_name = 'Small_grid_road.csv'
arcpy.conversion.TableToTable(out_statistic_table, out_path, out_name)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Small_grid_road.csv'>

In [106]:
temp_small_grid_sst = pd.read_csv(f'{out_path}/{out_name}')
id_list = np.unique(temp_small_grid_sst['ID'])
length_list = []
for temp_id in id_list:
    temp_length = temp_small_grid_sst[temp_small_grid_sst['ID'].isin([temp_id])]['Length_m'].values.sum()
    length_list.append(temp_length)
small_grid_sst = pd.DataFrame(columns=['ID', 'LENGTH'])
small_grid_sst['ID'] = id_list
small_grid_sst['LENGTH'] = length_list
small_grid_sst

,ID,LENGTH
0,96,2895.234708
1,253,1230.512143
2,259,636.760638
3,574,1409.475484
4,575,345.695546
...,...,...
992,23060,797.573076
993,23214,1907.584088
994,23215,9619.170725
995,23216,8996.086161


### 以完整的网格信息表(df)来作为基准进行排放因子统计

In [107]:
df

,OID_,ID,LON,LAT,NAME,square
0,0,1,101.576029,28.399021,康定县,9.000000
1,1,2,101.606922,28.399710,康定县,9.000000
2,2,3,101.637815,28.400390,康定县,9.000000
3,3,4,101.668708,28.401061,康定县,9.000000
4,4,5,101.699602,28.401724,康定县,9.000000
...,...,...,...,...,...,...
26776,26776,24645,106.391070,32.680985,汉中市,9.000000
26777,26777,24646,106.423582,32.680311,汉中市,9.000000
26778,26778,24647,106.456093,32.679627,汉中市,9.000000
26779,26779,24648,106.488604,32.678935,汉中市,7.737879


In [108]:
small_grid_sst

,ID,LENGTH
0,96,2895.234708
1,253,1230.512143
2,259,636.760638
3,574,1409.475484
4,575,345.695546
...,...,...
992,23060,797.573076
993,23214,1907.584088
994,23215,9619.170725
995,23216,8996.086161


In [109]:
big_grid_sst

,NAME,LENGTH
0,乐山市,1.927561e+05
1,仁寿县,1.703509e+05
2,内江市,1.439561e+05
3,华蓥市,2.229052e+04
4,南充县,3.387715e+05
5,宜宾市,2.377597e+05
6,广元市,5.299714e+04
7,德阳市,6.683858e+05
8,成都市,4.928331e+06
9,泸州市,1.532951e+05


In [127]:
result = pd.DataFrame(columns=['ID', 'LON', 'LAT', 'AREA', 'EF'])
ef_list = []
id_list = []
area_list = []
lon_list = []
lat_list = []
for temp_df in df.values:
    temp_id = temp_df[1]
    temp_area = temp_df[4]
    temp_sum = temp_df[5]
    temp_lon = temp_df[2]
    temp_lat = temp_df[3]
    if small_grid_sst['ID'].values.__contains__(temp_id):
        temp_length = small_grid_sst[small_grid_sst['ID'].isin([temp_id])]['LENGTH'].values[0]
        temp_total_length = big_grid_sst[big_grid_sst['NAME'].isin(['资阳市'])]['LENGTH'].values[0]
        temp_ef = temp_length/temp_total_length
    else:
        temp_ef = 0.0
        
    ef_list.append(temp_ef)
    id_list.append(temp_id)
    area_list.append(temp_area)
    lon_list.append(temp_lon)
    lat_list.append(temp_lat)
        
result['ID'] = id_list
result['LON'] = lon_list
result['LAT'] = lat_list
result['AREA'] = area_list
result['EF'] = ef_list

result

,ID,LON,LAT,AREA,EF
0,1,101.576029,28.399021,康定县,0.0
1,2,101.606922,28.399710,康定县,0.0
2,3,101.637815,28.400390,康定县,0.0
3,4,101.668708,28.401061,康定县,0.0
4,5,101.699602,28.401724,康定县,0.0
...,...,...,...,...,...
24644,24645,106.391070,32.680985,汉中市,0.0
24645,24646,106.423582,32.680311,汉中市,0.0
24646,24647,106.456093,32.679627,汉中市,0.0
24647,24648,106.488604,32.678935,汉中市,0.0


In [128]:
result_csv_path = 'F:\github\Haofan_Emission_Tools\output_directory\EF_TR_road_1.csv'
result.to_csv(result_csv_path, index=False)

In [130]:
result_point_path = 'F:\github\Haofan_Emission_Tools\output_directory\TR_road_1.shp'
arcpy.management.XYTableToPoint(result_csv_path, result_point_path, 'LON', 'LAT', z_field='EF')

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\TR_road_1.shp'>